In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from bs4 import BeautifulSoup
import requests
import pandas as pd
import os

In [ ]:
def DoorDash_Scraper(location):

    option = webdriver.ChromeOptions()
    option.add_argument("--window-size=3840,2160")
    #option.add_argument("--headless")

    driver = webdriver.Chrome('chromedriver.exe',options= option)

    driver.get('https://www.doordash.com/en-US')
    

    time.sleep(10)
    
    text = driver.page_source
    soup = BeautifulSoup(text, 'html.parser')


    elem = driver.find_element_by_class_name(soup.find('input').get('class')[0]) 
    time.sleep(3)
    elem.send_keys(location)
    time.sleep(2)
    elem.send_keys(Keys.RETURN)


    time.sleep(5)
    
    SCROLL_PAUSE_TIME = 5
    
    count = 0
    
    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    text = driver.page_source
    soup = BeautifulSoup(text, 'html.parser')

    rest_links = []

    rest = soup.find('div',{'data-anchor-id':'StoreLayoutListContainer'}).findAll('div',{'class':'sc-fHlXLc ddswzP'})
    for i in rest:
        rest_links.append('https://www.doordash.com/'+i.find('a').get('href'))
    print("Total Restaurants :- ", len(rest_links))

    Name = []
    Address = []
    City = []
    Postal_Code = []
    Menu_Item = []
    Item_Price = []
    Item_Image = []
    Menu_Description = []

    for link in rest_links:
        
        start_time = time.time()
        
        driver.get(link)
        time.sleep(10)

        text = driver.page_source
        soup = BeautifulSoup(text, 'html.parser')

        image_links = []
        items_names = []


        try:
            Name.append(soup.find('h1').text)
            name = soup.find('h1').text
            os.mkdir(name)
        except:
            Name.append('')
        try:
            Address.append(soup.find('div',{'class':'sc-cyQzhP fapFDy'}).findAll('span')[-1].text)
        except:
            Address.append('')
        try:
            City.append(soup.find('div',{'class':'sc-cyQzhP fapFDy'}).findAll('span')[-1].text.split(',')[1].strip())
        except:
            City.append('')
        try:
            Postal_Code.append(soup.find('div',{'class':'sc-cyQzhP fapFDy'}).findAll('span')[-1].text.split(',')[-2].split()[-1])
        except:
            Postal_Code.append('')
        try:
            Menu_Item.append(soup.findAll('div',{'class':'sc-izUgoq crnWTv'})[0].find('span',{'class':'sc-bdVaJa gImhEG'}).text)
        except:
            Menu_Item.append('')
        try:
            Menu_Description.append(soup.findAll('div',{'class':'sc-izUgoq crnWTv'})[0].find('span',{'class':'sc-sVRsr hrkYLh sc-bdVaJa huydyu'}).text)
        except:
            Menu_Description.append('')   
        try:
            Item_Price.append(soup.findAll('div',{'class':'sc-izUgoq crnWTv'})[0].find('span',{'class':'sc-bdVaJa hpbvJT'}).text)
        except:
            Item_Price.append('')
        try:
            Item_Image.append(soup.findAll('div',{'class':'sc-izUgoq crnWTv'})[0].find('img').get('src').replace('600','1200'))
            image_links.append(soup.findAll('div',{'class':'sc-izUgoq crnWTv'})[0].find('img').get('src').replace('600','1200'))
            items_names.append(soup.findAll('div',{'class':'sc-izUgoq crnWTv'})[0].find('span',{'class':'sc-bdVaJa gImhEG'}).text)
        except:
            Item_Image.append('')



        menu_data = soup.findAll('div',{'class':'sc-izUgoq crnWTv'})
        for i in range(1,len(menu_data)):
            try:
                Menu_Item.append(menu_data[i].find('span',{'class':'sc-bdVaJa gImhEG'}).text)
            except:
                Menu_Item.append('')
            try:
                Menu_Description.append(menu_data[i].find('span',{'class':'sc-sVRsr hrkYLh sc-bdVaJa huydyu'}).text)
            except:
                Menu_Description.append('')    
            try:
                Item_Price.append(menu_data[i].find('span',{'class':'sc-bdVaJa hpbvJT'}).text)
            except:
                Item_Price.append('')
            try:
                Item_Image.append(menu_data[i].find('img').get('src').replace('600','1200'))
                image_links.append(menu_data[i].find('img').get('src').replace('600','1200'))
                items_names.append(menu_data[i].find('span',{'class':'sc-bdVaJa gImhEG'}).text)
            except:
                Item_Image.append('')
            Address.append('')
            Postal_Code.append('')
            Name.append('')
            City.append('')





        try:
            for i in range(len(image_links)):
                response = requests.get(image_links[i])
                if response.status_code == 200:
                    with open('./'+name+'/'+items_names[i]+'.jpg', 'wb') as f:
                        f.write(response.content)
        except:
            pass

        Name  = Name[:len(Menu_Item)]
        Address  = Address[:len(Menu_Item)]
        Postal_Code  = Postal_Code[:len(Menu_Item)]
        City  = City[:len(Menu_Item)]
        
        
        count += 1
        print("Restaurant : {} - DONE, time - {} seconds ".format(count, int(time.time()-start_time)))
        
        

    df = pd.DataFrame({'Restaurant Name':Name,"Address":Address,"City":City,"Postal Code":Postal_Code,"Menu Item":Menu_Item,"Menu Description": Menu_Description,"Item Price":Item_Price,"Item_Image":Item_Image})
    df.to_excel(location.split(',')[0]+'_Data.xlsx')
    print('Scraper is run successfully check excel and images in folder')
    

    
    
DoorDash_Scraper('San Jose, CA, USA')